### Import and config

In [1]:
# Imports
import os
import time
import random
import json
import logging
import datetime

from dotenv import load_dotenv
from tqdm import tqdm

import pandas as pd
import requests

from sqlalchemy import create_engine, text, inspect, Text, Integer, BigInteger, Float, Boolean
from sqlalchemy.dialects.postgresql import JSONB

# Configuration
load_dotenv()

# API urls
AUTH_URL = os.getenv('AUTH_URL')
ACTIVITIES_URL = os.getenv('ACTIVITIES_URL')
ACTIVITY_DETAIL_URL_TMPL = os.getenv('ACTIVITY_DETAIL_URL_TMPL')
ACTIVITY_KUDOS_URL_TMPL = os.getenv('ACTIVITY_KUDOS_URL_TMPL')
ACTIVITY_ZONES_URL_TMPL = os.getenv('ACTIVITY_ZONES_URL_TMPL')
SEGMENTS_URL_TMPL = os.getenv('SEGMENTS_URL_TMPL')

# API keys
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

# DB
DB_URI = os.getenv('DB_URI')

# Bronze tables
TARGET_B_SCHEMA = os.getenv('TARGET_B_SCHEMA')
ACTIVITIES_B_TABLE = os.getenv('ACTIVITIES_B_TABLE')
DETAILS_B_TABLE = os.getenv('DETAILS_B_TABLE')
KUDOS_B_TABLE = os.getenv('KUDOS_B_TABLE')
ZONES_B_TABLE = os.getenv('ZONES_B_TABLE')
SEGMENTS_DETAILS_B_TABLE = os.getenv('SEGMENTS_DETAILS_B_TABLE')

# Pagination
PER_PAGE = int(os.getenv('PER_PAGE'))
MAX_PAGES = int(os.getenv('MAX_PAGES'))

# Timeouts and retries
REQUEST_TIMEOUT = int(os.getenv('REQUEST_TIMEOUT'))
MAX_RETRIES = int(os.getenv('MAX_RETRIES'))
BASE_SLEEP = float(os.getenv('BASE_SLEEP'))

REFRESH_THRESHOLD_DAYS = int(os.getenv('REFRESH_THRESHOLD_DAYS'))

# Other
LOG_LEVEL = os.getenv('LOG_LEVEL')

logging.basicConfig(
    level=getattr(logging, LOG_LEVEL.upper(), logging.INFO),
    format="%(asctime)s | %(levelname)s | %(message)s"
)

pd.set_option('display.max_columns', None)

UPDATE_SEGMENT = True

### API keys validation

In [2]:
REQUIRED_API_ENV = ['CLIENT_ID', 'CLIENT_SECRET', 'REFRESH_TOKEN']
missing_api_env = [env for env in REQUIRED_API_ENV if not os.getenv(env)]
if missing_api_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_api_env)}.")

### DB names validation

In [3]:
REQUIRED_DB_ENV = ['DB_URI', 'TARGET_B_SCHEMA', 'ACTIVITIES_B_TABLE', 'DETAILS_B_TABLE', 'ZONES_B_TABLE', 'SEGMENTS_DETAILS_B_TABLE']
missing_db_env = [env for env in REQUIRED_DB_ENV if not os.getenv(env)]
if missing_db_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_db_env)}.")

### Connecting with PostgreSQL

In [4]:
engine = create_engine(
  DB_URI, 
  pool_pre_ping=True, 
  pool_size=5, 
  max_overflow=10
)
with engine.connect() as conn:
  conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_B_SCHEMA};"))
logging.info(f"Connection established and {TARGET_B_SCHEMA} schema exists.")

2025-12-09 12:53:22,349 | INFO | Connection established and bronze schema exists.


### Token authorization

In [5]:
def get_access_token(auth_url: str, client_id: str, client_secret: str, refresh_token: str, timeout: int = 30) -> str:
  """
  Request a new access token from an OAuth2 authentication endpoint.

  The function uses a refresh token to obtain a short-lived access token. 
  If the response does not contain an ``access_token``, a RuntimeError is raised.

  Parameters
  ----------
  auth_url : str
      URL of the OAuth2 token endpoint.
  client_id : str
      OAuth2 client identifier.
  client_secret : str
      OAuth2 client secret.
  refresh_token : str
      Refresh token used to request a new access token.
  timeout : int, default=30
      Timeout in seconds for the HTTP request.

  Returns
  -------
  str
      The access token string retrieved from the authentication server.

  Raises
  ------
  RuntimeError
      If the response does not include an ``access_token``.
  requests.exceptions.RequestException
      If the HTTP request fails (e.g., network error, timeout).

  Notes
  -----
  Logs an informational message when the token is successfully retrieved.
  """

  payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': 'refresh_token',
  }
  res = requests.post(auth_url, data=payload, timeout=timeout)
  data = res.json()
  token = data.get('access_token')
  if not token:
    raise RuntimeError(f"No access token in response: {data}")
  logging.info('Access token retrived.')
  return token

access_token = get_access_token(AUTH_URL, CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)

2025-12-09 12:53:22,653 | INFO | Access token retrived.


### HTTP session

In [6]:
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {access_token}"})

def get_json_with_retry(url: str, params=None, max_retries: int = MAX_RETRIES, timeout: int = REQUEST_TIMEOUT, base_sleep: float = BASE_SLEEP):
  """
  Send a GET request with automatic retries and return the parsed JSON response.

  The function retries on:
    * HTTP 429 (rate limiting) — respects the ``Retry-After`` header if present,
      otherwise waits an increasing backoff time.
    * HTTP 5xx errors — retries with exponential backoff and jitter.
    * Network/connection errors — retries with exponential backoff and jitter.

  For each attempt, the backoff time increases by ``base_sleep * attempt`` seconds
  plus a small random jitter. On the final attempt, any error is raised.

  Parameters
  ----------
  url : str
      Endpoint URL to send the GET request to.
  params : dict, optional
      Query string parameters to include in the request.
  max_retries : int, default=MAX_RETRIES
      Maximum number of retry attempts before failing.
  timeout : int, default=REQUEST_TIMEOUT
      Timeout in seconds for each HTTP request.
  base_sleep : float, default=BASE_SLEEP
      Base number of seconds used for exponential backoff between retries.

  Returns
  -------
  dict
      Parsed JSON response from the server.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after all retry attempts.
  requests.exceptions.HTTPError
      If the server returns a 4xx/5xx response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * HTTP 429 triggers a wait using ``Retry-After`` if available, otherwise a fallback.
  * Logs warnings before each retry and errors if JSON parsing fails.
  * Uses a global ``requests.Session`` (`session`) for connection pooling.
  """

  for attempt in range(1, max_retries + 1):
    try:
      resp = session.get(url, params=params, timeout=timeout)
      
      if resp.status_code == 429:
        retry_after = resp.headers.get('Retry-After')
        if retry_after and retry_after.isdigit():
          sleep_for = int(retry_after)
        else:
          sleep_for = max(base_sleep * attempt, 15)
        logging.warning(f"HTTP 429 - wait {sleep_for}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      if 500 <= resp.status_code < 600:
        if attempt == max_retries:
          resp.raise_for_status()
        sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
        logging.warning(f"HTTP {resp.status_code} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      resp.raise_for_status()

      try:
        return resp.json()
      except json.JSONDecodeError:
        logging.error('JSON parsing error')
        raise
    
    except requests.exceptions.RequestException as e:
      if attempt == max_retries:
        logging.exception('Request error (last attempt)')
        raise
      sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
      logging.warning(f"{e} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
      time.sleep(sleep_for)

### Getting activities list

In [7]:
def fetch_all_activities(activites_url: str, per_page: int = PER_PAGE, max_pages: int = MAX_PAGES):
  """
  Fetch all activity records from a paginated API endpoint.

  The function iterates through API pages until either:
    * the maximum number of pages is reached (``max_pages``), or
    * the API returns an empty list (end of results).

  Each page is retrieved using ``get_json_with_retry`` to ensure resilience
  against transient errors (rate limits, timeouts, 5xx responses).

  Parameters
  ----------
  activites_url : str
      The base URL of the activities endpoint (must support ``per_page`` and ``page`` query params).
  per_page : int, default=PER_PAGE
      Number of activity records to request per page.
  max_pages : int, default=MAX_PAGES
      Maximum number of pages to fetch before stopping.

  Returns
  -------
  list of dict
      Combined list of activity objects returned by the API.

  Raises
  ------
  RuntimeError
      If the API response is not a list (unexpected schema).
  requests.exceptions.RequestException
      If the underlying HTTP requests fail after retries.

  Notes
  -----
  * Logs the number of records downloaded per page and the running total.
  * Stops early if the API returns an empty list before reaching ``max_pages``.
  * The total number of records is ``per_page * n_pages`` at most.
  """

  all_items = []
  page = 1
  while page <= max_pages:
    params = {'per_page': per_page, 'page': page}
    data = get_json_with_retry(activites_url, params=params)

    if not isinstance(data, list):
      raise RuntimeError(f"Unexpected response type for page {page}: {type(data)} — expected list")
    
    if not data:
      break
    
    all_items.extend(data)
    logging.info(f"Page {page}: downloaded {len(data)} records (total: {len(all_items)})")
    page += 1
  return all_items

activities_raw = fetch_all_activities(ACTIVITIES_URL, per_page=PER_PAGE, max_pages=MAX_PAGES)
logging.info(f"Total activities downloaded: {len(activities_raw)}")
activities_df = pd.json_normalize(activities_raw, sep='_')
if activities_df.empty:
    raise RuntimeError('No activities to save.')
activities_df.head()

2025-12-09 12:53:28,577 | INFO | Page 1: downloaded 200 records (total: 200)
2025-12-09 12:53:35,769 | INFO | Page 2: downloaded 200 records (total: 400)
2025-12-09 12:53:43,267 | INFO | Page 3: downloaded 200 records (total: 600)
2025-12-09 12:53:49,016 | INFO | Page 4: downloaded 200 records (total: 800)
2025-12-09 12:53:54,184 | INFO | Page 5: downloaded 200 records (total: 1000)
2025-12-09 12:53:56,637 | INFO | Page 6: downloaded 175 records (total: 1175)
2025-12-09 12:53:56,867 | INFO | Total activities downloaded: 1175


,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,device_name,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,athlete_id,athlete_resource_state,map_id,map_summary_polyline,map_resource_state,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,average_temp
0,2,Morning Yoga,0.0,4537,4537,0.0,Yoga,Yoga,31.0,Garmin Forerunner 970,16691408683,2025-12-09T07:02:45Z,2025-12-09T08:02:45Z,(GMT+01:00) Africa/Algiers,3600.0,None,None,None,0,1,0,1,0,True,False,False,False,everyone,False,None,[],[],0.000,0.00,True,77.4,123.0,False,True,0.0,0.0,1.778110e+10,17781098305,garmin_ping_510724236712,False,0,0,False,6.0,81055898,1,a16691408683,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6.5km Easy Run🙂‍↔️,6565.0,2317,2355,62.0,Run,Run,NaN,Garmin Forerunner 970,16684277150,2025-12-08T12:45:47Z,2025-12-08T13:45:47Z,(GMT+01:00) Europe/Warsaw,3600.0,None,None,None,4,8,0,1,0,False,False,False,False,everyone,False,g24134620,"[52.755966, 15.226957]","[52.756339, 15.228833]",2.833,3.80,True,142.8,155.0,False,True,60.4,21.6,1.777387e+10,17773866522,garmin_ping_510442524516,False,3,0,False,25.0,81055898,1,a16684277150,a|~`I{`}{ACa@I_@Gu@GgBOgAYoC_@wCCc@EY?GRSDM@y@...,2,84.3,351.5,536.0,347.0,True,814.6,NaN
2,2,Drop Set Hill Reps🏡,9222.0,3286,3292,194.0,Run,Run,3.0,Garmin Forerunner 970,16678081919,2025-12-07T16:57:40Z,2025-12-07T17:57:40Z,(GMT+01:00) Europe/Warsaw,3600.0,None,None,None,1,9,0,1,0,False,False,False,False,everyone,False,g24134620,"[52.755944, 15.226775]","[52.755707, 15.225744]",2.806,4.50,True,148.6,173.0,False,True,53.0,23.4,1.776756e+10,17767559810,garmin_ping_510165492006,False,0,0,False,51.0,81055898,1,a16678081919,i|~`Io{}{Az@]PMb@QzCiAf@_@nBo@`@SVGNCP@p@?Xa@F...,2,82.2,352.8,765.0,369.0,True,1160.4,NaN
3,2,24km Long Run🇪🇸,24159.5,8178,9074,61.0,Run,Run,2.0,Garmin Forerunner 970,16663625820,2025-12-06T07:19:02Z,2025-12-06T08:19:02Z,(GMT+01:00) Europe/Warsaw,3600.0,None,None,None,15,8,0,1,0,False,False,False,False,everyone,False,g23642256,"[51.107712, 17.124042]","[51.107462, 17.124548]",2.954,4.54,True,143.5,155.0,False,True,125.0,112.2,1.775284e+10,17752839303,garmin_ping_509697762763,False,7,0,False,92.0,81055898,1,a16663625820,wr}vHwbngBsO`WaHbJg@~@Np@j@nAnGrMz@pATnA?p@OrB...,2,85.8,344.4,507.0,327.0,True,2818.5,NaN
4,2,12km Easy Run😋,12328.4,4326,4366,42.0,Run,Run,NaN,Garmin Forerunner 970,16648928719,2025-12-04T14:20:16Z,2025-12-04T15:20:16Z,(GMT+01:00) Europe/Warsaw,3600.0,None,None,None,9,8,0,1,0,False,False,False,False,everyone,False,g24134620,"[51.107732, 17.123874]","[51.107959, 17.123598]",2.850,3.48,True,142.2,152.0,False,True,120.6,106.6,1.773790e+10,17737901963,garmin_ping_509108665098,False,6,0,False,43.0,81055898,1,a16648928719,as}vHmbngB}AjCeBhCmBdDa@j@c@z@_DbFmE~FqA|AYj@?...,2,85.1,322.6,411.0,321.0,True,1396.4,NaN


### Set `bronze.activities` types map

In [8]:
activities_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_summary_polyline": Text,
    "map_resource_state": Integer,
    "average_temp": Float,
}

### Save `bronze.activities` to database

In [9]:
table_full_name = f"{TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"
logging.warning(f"Whole table {table_full_name} will be overwritten.")
activities_df.to_sql(
  ACTIVITIES_B_TABLE, 
  engine, 
  schema=TARGET_B_SCHEMA, 
  if_exists="replace", 
  index=False, 
  method="multi", 
  chunksize=1000, 
  dtype=activities_dtype_map
)

2025-12-09 12:53:56,938 | WARNING | Whole table bronze.activities will be overwritten.


-2

### Identify missing details — only `id` that are not present in `bronze.activities_details`

In [10]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} (id BIGINT PRIMARY KEY)
"""))
  existing_ids = pd.read_sql(text(f"SELECT id FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)
  existing_ids_set = set(existing_ids['id'].astype('Int64').dropna().to_list())

all_ids_set = set(activities_df['id'].astype('Int64').dropna().to_list())
missing_ids = sorted(all_ids_set - existing_ids_set)
logging.info(f"Missing details of {len(missing_ids)} IDs")
pd.DataFrame({'id': missing_ids}).head()

2025-12-09 12:53:58,372 | INFO | Missing details of 4 IDs


,id
0,16663625820
1,16678081919
2,16684277150
3,16691408683


### Identify recent activities — update and download kudos of `id` from the last 30 days

In [11]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} (id BIGINT PRIMARY KEY)
"""))
  bronze_activities_df = pd.read_sql(text(f"SELECT * FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

In [12]:
bronze_activities_df['date'] =  pd.to_datetime(bronze_activities_df['start_date_local']).dt.tz_localize(None)
today = pd.Timestamp('today').normalize()
bronze_activities_df['activity_age_days'] = (today - bronze_activities_df['date'].dt.normalize()).dt.days

In [13]:
recent_activities_df = bronze_activities_df[bronze_activities_df['activity_age_days'] <= REFRESH_THRESHOLD_DAYS]
recent_activities_ids = set(recent_activities_df['id'].astype('Int64').dropna().to_list())
recent_activities_ids = sorted(recent_activities_ids)
logging.info(f"Refreshing details of {len(recent_activities_ids)} IDs")
pd.DataFrame({'id': recent_activities_ids}).head()

2025-12-09 12:53:58,759 | INFO | Refreshing details of 27 IDs


,id
0,16404369707
1,16426332819
2,16436098407
3,16446595964
4,16462518933


### Download activities details

In [14]:
def fetch_activity_details(activity_id: int):
  """
  Fetch detailed information for a single activity.

  Builds the activity detail endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity details as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_DETAIL_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_DETAIL_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)

ids_to_be_downloaded = sorted(set(missing_ids) | set(recent_activities_ids))
details_records = []
for i, act_id in tqdm(enumerate(ids_to_be_downloaded, start=1), total=len(ids_to_be_downloaded)):
  try:
    resp = fetch_activity_details(act_id)
    
    if not isinstance(resp, dict):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue
    details_records.append(resp)
    
  except Exception as e:
    logging.error(f"Error downloading details for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"Details downloaded: {len(details_records)} / {len(ids_to_be_downloaded)}")
details_df_new = pd.json_normalize(details_records, sep='_')

100%|██████████| 31/31 [04:21<00:00,  8.44s/it]
2025-12-09 12:58:20,291 | INFO | Details downloaded: 31 / 31


In [15]:
if not details_df_new.empty:
  details_df_new = details_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  details_df_new = details_df_new.dropna(how='all')
  
details_df_new.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,device_name,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,splits_metric,splits_standard,laps,best_efforts,stats_visibility,hide_from_home,embed_token,available_zones,athlete_id,athlete_resource_state,map_id,map_polyline,map_resource_state,map_summary_polyline,gear_primary,gear_name,gear_nickname,gear_resource_state,gear_retired,gear_distance,gear_converted_distance,photos_primary,photos_count,similar_activities_effort_count,similar_activities_average_speed,similar_activities_min_average_speed,similar_activities_mid_average_speed,similar_activities_max_average_speed,similar_activities_pr_rank,similar_activities_frequency_milestone,similar_activities_trend_speeds,similar_activities_trend_current_activity_index,similar_activities_trend_min_speed,similar_activities_trend_mid_speed,similar_activities_trend_max_speed,similar_activities_trend_direction,similar_activities_resource_state,photos_primary_unique_id,photos_primary_urls_600,photos_primary_urls_100,photos_primary_source,photos_primary_media_type,photos_use_primary_photo,private_note
0,3,Taper 400s😮‍💨,8332.3,2813,3404,5.0,Run,Run,3.0,Garmin Forerunner 970,16404369707,2025-11-09T12:06:56Z,2025-11-09T13:06:56Z,(GMT+01:00) Europe/Warsaw,3600.0,None,None,None,1,9,1,1,0,False,False,False,False,everyone,False,g23642256,"[51.110464, 17.076196]","[51.110845, 17.077839]",2.962,4.80,81.3,335.5,545.0,342.0,True,944.3,True,147.3,172.0,False,True,111.6,108.0,17490394286,17490394286,garmin_ping_500113668336,False,0,0,False,40.0,Elegancko przepalona noga i każda czterysetka🙂...,639.0,None,None,"[{'id': 3422588680517569592, 'resource_state':...","[{'distance': 1002.8, 'elapsed_time': 356, 'el...","[{'distance': 1609.5, 'elapsed_time': 572, 'el...","[{'id': 58471050832, 'resource_state': 2, 'nam...","[{'id': 69506026688, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,6ac220c2cac901deceacebe109ad7b686bd27eda,"[heartrate, pace, power]",81055898,1,a16404369707,ko}vHeefgBXEFCPQJONm@@e@Ac@_@_CGUMWWQWGM@SJWXM...,3,ko}vHeefgBr@[Z}@?iAg@uCe@i@o@?e@d@QdA^hDL`@h@h...,False,Adidas EVO SL,None,2.0,False,345476.0,345.5,NaN,0,3.0,2.998500,2.962069,2.990476,3.019433,3.0,3.0,"[3.019432918395574, 3.0163788612180227, 2.9980...",2.0,2.962069,2.990476,3.019433,-1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,35. Bieg Niepodległości🇵🇱,10036.4,2703,2703,25.0,Run,Run,1.0,Garmin Forerunner 970,16426332819,2025-11-11T10:20:13Z,2025-11-11T11:20:13Z,(GMT+01:00) Europe/Warsaw,3600.0,None,None,None,0,17,3,234,0,False,False,False,False,everyone,False,g27111424,"[52.250014, 20.986712]","[52.249995, 20.986419]",3.713,5.06,89.3,439.8,580.0,439.0,True,1189.0,True,171.0,178.0,False,True,120.2,103.8,17512522294,17512522294,garmin_ping_500908993574,False,0,2,False,117.0,"Wszystko fajnie, ale szkoda tej sekundki🫡\nNet...",750.0,None,None,"[{'id': 3423367781034818888, 'resource_state':...","[{'distance': 1003.5, 'elapsed_time': 264, 'el...","[{'distance': 1611.4, 'elapsed_time': 426, 'el...","[{'id': 58548246007, 'resource_state': 2, 'nam...","[{'id': 69136476785, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,787c4547d7b38eddb8577760e21889c9a606462e,"[heartrate, pace, power]",81055898,1,a16426332819,qa|}H}}a_C~AmA|@u@rAqArAiApDgDpA{@hB}AtCyBfC{B...,3,q

### Set `bronze.activities_details` types map

In [16]:
activities_details_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "perceived_exertion": Text,
    "prefer_perceived_exertion": Text,
    "segment_efforts": JSONB,
    "splits_metric": JSONB,
    "splits_standard": JSONB,
    "laps": JSONB,
    "best_efforts": JSONB,
    "stats_visibility": JSONB,
    "hide_from_home": Boolean,
    "device_name": Text,
    "embed_token": Text,
    "available_zones": JSONB,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_polyline": Text,
    "map_resource_state": Integer,
    "map_summary_polyline": Text,
    "gear_primary": Boolean,
    "gear_name": Text,
    "gear_nickname": Text,
    "gear_resource_state": Float,
    "gear_retired": Boolean,
    "gear_distance": Float,
    "gear_converted_distance": Float,
    "photos_primary": JSONB,
    "photos_count": Integer,
    "similar_activities_effort_count": Float,
    "similar_activities_average_speed": Float,
    "similar_activities_min_average_speed": Float,
    "similar_activities_mid_average_speed": Float,
    "similar_activities_max_average_speed": Float,
    "similar_activities_pr_rank": Float,
    "similar_activities_frequency_milestone": Float,
    "similar_activities_trend_speeds": JSONB,
    "similar_activities_trend_current_activity_index": Float,
    "similar_activities_trend_min_speed": Float,
    "similar_activities_trend_mid_speed": Float,
    "similar_activities_trend_max_speed": Float,
    "similar_activities_trend_direction": Float,
    "similar_activities_resource_state": Float,
    "average_temp": Float,
    "photos_primary_unique_id": Text,
    "photos_primary_urls_600": Text,
    "photos_primary_urls_100": Text,
    "photos_primary_source": Integer,
    "photos_primary_media_type": Integer,
    "photos_use_primary_photo": Boolean,
    "private_note": Text
}

### Save `bronze.activities_details` to database

In [17]:
if details_df_new.empty:
  logging.info('No new details to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA):
    details_df_new.head(0).to_sql(
      DETAILS_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_details_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE details_stg
        AS TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    details_df_new.to_sql(
      "details_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_details_dtype_map
      )
    
    # Prepare upsert
    stg_cols = details_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in details_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM details_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS details_stg;'))
  
    logging.info('Activities details saved to PostgreSQL.')

2025-12-09 12:58:20,606 | INFO | Activities details saved to PostgreSQL.


### Sanity check

In [18]:
with engine.begin() as conn:
    total_acts = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"), conn)
    total_det = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}: {int(total_acts['n'][0])}")
logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}: {int(total_det['n'][0])}")

2025-12-09 12:58:20,620 | INFO | Total number of records in bronze.activities: 1175
2025-12-09 12:58:20,620 | INFO | Total number of records in bronze.activities_details: 1175


### Identify missing kudos — only `activity_id` that are not present in `bronze.kudos`

In [19]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} (id TEXT PRIMARY KEY, activity_id BIGINT)
"""))
  existing_ids_kudos = pd.read_sql(text(f"SELECT activity_id FROM {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}"), conn)
  existing_ids_kudos_set = set(existing_ids_kudos['activity_id'].astype('Int64').dropna().to_list())

all_ids_kudos_set = set(activities_df[activities_df['kudos_count'] > 0]['id'].astype('Int64').dropna().to_list())
missing_ids_kudos = sorted(all_ids_kudos_set - existing_ids_kudos_set, reverse=True)
logging.info(f"Missing kudos of {len(missing_ids_kudos)} activities")
pd.DataFrame({'activity_id': missing_ids_kudos}).head()

2025-12-09 12:58:20,647 | INFO | Missing kudos of 4 activities


,activity_id
0,16691408683
1,16684277150
2,16678081919
3,16663625820


### Download activities kudos

In [20]:
def fetch_activity_kudos(activity_id: int):
  """
  Fetch all kudos for a single activity.

  Builds the activity kudos endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity kudos as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_KUDOS_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_KUDOS_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)

kudos_to_be_downloaded = sorted(set(missing_ids_kudos) | set(recent_activities_ids))

kudos_records = []
for i, act_id in tqdm(enumerate(kudos_to_be_downloaded, start=1), total=len(kudos_to_be_downloaded)):
  try:
    resp = fetch_activity_kudos(act_id)
    
    if not isinstance(resp, list):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue

    for i, kudos in enumerate(resp):
      kudos["activity_id"] = act_id
      kudos["kudos_id"] = i
    kudos_records.extend(resp)
    
  except Exception as e:
    logging.error(f"Error downloading kudos for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"Kudos downloaded: {pd.DataFrame(kudos_records)['activity_id'].nunique()} / {len(kudos_to_be_downloaded)}")


100%|██████████| 31/31 [04:44<00:00,  9.16s/it]
2025-12-09 13:03:04,775 | INFO | Kudos downloaded: 31 / 31


In [21]:

kudos_df_new = pd.json_normalize(kudos_records, sep='_')
if not kudos_df_new.empty:
  kudos_df_new = kudos_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  kudos_df_new = kudos_df_new.dropna(how='all')
  
kudos_df_new['id'] = kudos_df_new[['activity_id', 'kudos_id']].astype("string").agg("-".join, axis=1)

In [22]:
kudos_df_new.head()

,resource_state,firstname,lastname,activity_id,kudos_id,id
0,2,Ola,Ł.,16404369707,0,16404369707-0
1,2,Kacper,K.,16404369707,1,16404369707-1
2,2,Alicja,Ł.,16404369707,2,16404369707-2
3,2,Filip,C.,16404369707,3,16404369707-3
4,2,Agnieszka,G.,16404369707,4,16404369707-4


### Set `bronze.kudos` types map

In [23]:
activities_kudos_dtype_map = {
    "resource_state": Integer,
    "first_name": Text,
    "last_name": Float,
    "activity_id": BigInteger,
    "id": Text
}

### Save `bronze.kudos` to database

In [24]:
if kudos_df_new.empty:
  logging.info('No new kudos to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=KUDOS_B_TABLE, schema=TARGET_B_SCHEMA):
    kudos_df_new.head(0).to_sql(
      KUDOS_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_kudos_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=KUDOS_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE kudos_stg
        AS TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    kudos_df_new.to_sql(
      "kudos_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_kudos_dtype_map
      )
    
    # Prepare upsert
    stg_cols = kudos_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in kudos_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM kudos_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS kudos_stg;'))
  
    logging.info('Activities kudos saved to PostgreSQL.')

2025-12-09 13:03:04,957 | INFO | Activities kudos saved to PostgreSQL.


### Identify missing zones — only `activity_id` that are not present in `bronze.activities_zones`

In [25]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{ZONES_B_TABLE} (id TEXT PRIMARY KEY, activity_id BIGINT)
"""))
  existing_ids_zones = pd.read_sql(text(f"SELECT activity_id FROM {TARGET_B_SCHEMA}.{ZONES_B_TABLE}"), conn)
  existing_ids_zones_set = set(existing_ids_zones['activity_id'].astype('Int64').dropna().to_list())

all_ids_zones_set = set(activities_df['id'].astype('Int64').dropna().to_list())
missing_ids_zones = sorted(all_ids_zones_set - existing_ids_zones_set, reverse=True)
logging.info(f"Missing zones of {len(missing_ids_zones)} activities")
pd.DataFrame({'activity_id': missing_ids_zones}).head()

2025-12-09 13:03:04,991 | INFO | Missing zones of 4 activities


,activity_id
0,16691408683
1,16684277150
2,16678081919
3,16663625820


### Download activities zones

In [26]:
def fetch_activity_zones(activity_id: int):
  """
  Fetch all zones for a single activity.

  Builds the activity zones endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity zones as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_ZONES_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_ZONES_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)


zones_records = []
for i, act_id in tqdm(enumerate(missing_ids_zones, start=1), total=len(missing_ids_zones)):
  try:
    resp = fetch_activity_zones(act_id)
    
    if not isinstance(resp, list):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue

    if not resp:
      resp = [{'score': None,
              'distribution_buckets': None,
              'type': 'dummy',
              'resource_state': None,
              'sensor_based': None,
              'points': None,
              'custom_zones': None}]

    for i, zones in enumerate(resp):
      zones["activity_id"] = act_id
    zones_records.extend(resp)
    
  except Exception as e:
    logging.error(f"Error downloading zones for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"zones downloaded: {pd.DataFrame(zones_records)['activity_id'].nunique()} / {len(missing_ids_zones)}")

100%|██████████| 4/4 [00:37<00:00,  9.30s/it]
2025-12-09 13:03:42,215 | INFO | zones downloaded: 4 / 4


In [27]:
resp_test = [{'score': None,
              'distribution_buckets': None,
              'type': None,
              'resource_state': None,
              'sensor_based': None,
              'points': None,
              'custom_zones': None}]

In [28]:

if zones_records:
  zones_df_new = pd.json_normalize(zones_records, sep='_')
else:
  zones_df_new = pd.DataFrame()

if not zones_df_new.empty:
  zones_df_new = zones_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  zones_df_new = zones_df_new.dropna(how='all')
  zones_df_new['id'] = zones_df_new[['activity_id', 'type']].astype("string").agg("-".join, axis=1)

In [29]:
zones_df_new.head()

,score,distribution_buckets,type,resource_state,sensor_based,points,custom_zones,activity_id,id
0,6.0,"[{'min': 0, 'max': 133, 'time': 4537.0}, {'min...",heartrate,3,True,0.0,True,16691408683,16691408683-heartrate
1,25.0,"[{'min': 0, 'max': 133, 'time': 80.0}, {'min':...",heartrate,3,True,0.0,True,16684277150,16684277150-heartrate
2,4.0,"[{'max': 2.828, 'min': 0, 'time': 684.0}, {'ma...",pace,3,True,NaN,NaN,16684277150,16684277150-pace
3,NaN,"[{'max': 0, 'min': 0, 'time': 0.0}, {'max': 50...",power,3,True,NaN,NaN,16684277150,16684277150-power
4,51.0,"[{'min': 0, 'max': 133, 'time': 196.0}, {'min'...",heartrate,3,True,11.0,True,16678081919,16678081919-heartrate


### Download segment details

In [30]:
with engine.begin() as conn:
  segment_efforts_df = pd.read_sql(text(f"SELECT segment_efforts FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

In [31]:
segment_efforts_df = segment_efforts_df.explode('segment_efforts', ignore_index=True).dropna()

In [32]:
segment_efforts_df = pd.json_normalize(segment_efforts_df['segment_efforts'])

In [33]:
running_segment_efforts_df = segment_efforts_df[segment_efforts_df['segment.activity_type'] == 'Run']

In [34]:
distinct_running_segment_efforts_df = running_segment_efforts_df[['segment.id']].drop_duplicates()

In [35]:
distinct_running_segment_efforts_df.head()

,segment.id
76,24333805
77,24333819
78,25769916
79,39787857
80,37929254


In [36]:
segment_details_ids_set = set(distinct_running_segment_efforts_df['segment.id'].astype('Int64').dropna().to_list())

In [37]:
def fetch_segment_details(segment_id: int):
  """
  Fetch detailed information for a single segment.

  Builds the segment detail endpoint URL using the given segment ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  segment_id : int
      Unique identifier of the segment.

  Returns
  -------
  dict
      JSON object containing the segment details as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``segment_DETAIL_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = SEGMENTS_URL_TMPL.format(id=segment_id)
  return get_json_with_retry(url, params=None)

ids_to_be_downloaded = segment_details_ids_set
segment_records = []

if UPDATE_SEGMENT:

  for i, seg_id in tqdm(enumerate(ids_to_be_downloaded, start=1), total=len(ids_to_be_downloaded)):
    try:
      resp = fetch_segment_details(seg_id)
      
      if not isinstance(resp, dict):
        logging.warning(f"id={seg_id}: unexpected response type ({type(resp)}), skip")
        continue
      segment_records.append(resp)
      
    except Exception as e:
      logging.error(f"Error downloading details for id={seg_id}: {e}")
    
    time.sleep(random.randint(7, 9))

logging.info(f"Details downloaded: {len(segment_records)} / {len(ids_to_be_downloaded)}")
segments_df_new = pd.json_normalize(segment_records, sep='_')

 27%|██▋       | 177/646 [25:43<1:02:18,  7.97s/it]2025-12-09 13:29:26,689 | WARNING | HTTP 429 - wait 15s (attempt 1/5)
2025-12-09 13:29:42,040 | WARNING | HTTP 429 - wait 16.0s (attempt 2/5)
2025-12-09 13:29:58,291 | WARNING | HTTP 429 - wait 24.0s (attempt 3/5)
 43%|████▎     | 277/646 [40:49<53:30,  8.70s/it]  2025-12-09 13:44:32,362 | WARNING | HTTP 429 - wait 15s (attempt 1/5)
2025-12-09 13:44:47,574 | WARNING | HTTP 429 - wait 16.0s (attempt 2/5)
 58%|█████▊    | 377/646 [55:18<36:36,  8.16s/it]  2025-12-09 13:59:01,695 | WARNING | HTTP 429 - wait 15s (attempt 1/5)
2025-12-09 13:59:16,889 | WARNING | HTTP 429 - wait 16.0s (attempt 2/5)
2025-12-09 13:59:33,340 | WARNING | HTTP 429 - wait 24.0s (attempt 3/5)
2025-12-09 13:59:57,532 | WARNING | HTTP 429 - wait 32.0s (attempt 4/5)
 74%|███████▍  | 477/646 [1:11:00<24:28,  8.69s/it]2025-12-09 14:14:43,188 | WARNING | HTTP 429 - wait 15s (attempt 1/5)
2025-12-09 14:14:58,408 | WARNING | HTTP 429 - wait 16.0s (attempt 2/5)
 89%|███████

In [38]:
segments_df_new = pd.json_normalize(segment_records, sep='_')

In [39]:
segments_df_new.head()

,id,resource_state,name,activity_type,distance,average_grade,maximum_grade,elevation_high,elevation_low,start_latlng,end_latlng,elevation_profile,climb_category,city,state,country,private,hazardous,starred,created_at,updated_at,total_elevation_gain,effort_count,athlete_count,star_count,elevation_profiles_light_url,elevation_profiles_dark_url,map_id,map_polyline,map_resource_state,athlete_segment_stats_pr_elapsed_time,athlete_segment_stats_pr_date,athlete_segment_stats_pr_visibility,athlete_segment_stats_pr_activity_id,athlete_segment_stats_pr_activity_visibility,athlete_segment_stats_effort_count,xoms_kom,xoms_qom,xoms_overall,xoms_destination_href,xoms_destination_type,xoms_destination_name,local_legend_athlete_id,local_legend_title,local_legend_profile,local_legend_effort_description,local_legend_effort_count,local_legend_effort_counts_overall,local_legend_effort_counts_female,local_legend_destination,local_legend
0,19517444,3,BRZEŹNO-SOPOT,Run,3550.0,0.1,6.1,8.4,1.2,"[54.409473, 18.636221]","[54.428292, 18.593357]",https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,0,Gdańsk,pomorskie,Poland,False,False,False,2018-12-20T11:44:06Z,2021-05-19T08:04:48Z,10.2,14917,4655,1,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,s19517444,ezakIk{vpBe@~BWnBYlAe@jCs@tCc@rBy@hE{@bGYhDOj@...,3,1222,2024-11-23,everyone,12964095995,everyone,1,11:32,14:48,11:32,strava://segments/19517444/leaderboard,overall,All-Time,152263214.0,Igor Kania,https://dgalywyr863hv.cloudfront.net/pictures/...,24 efforts in the last 90 days,24,24 efforts,13 efforts,strava://segments/19517444/local_legend?catego...,NaN
1,34463750,3,Aleja Jana Pawła II / John Paul II Avenue,Run,897.0,-0.0,1.3,115.8,114.7,"[52.235861, 20.997782]","[52.228274, 21.002288]",https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,0,Warsaw,Masovian Voivodeship,Poland,False,False,False,2023-05-23T10:03:44Z,2023-05-23T10:20:35Z,0.0,21810,14546,0,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,s34463750,ciy}Hccd_CxBUj@PnA_@f@Gd@Ob@W`@a@J@hAi@Vk@Hs@B...,3,246,2025-11-11,everyone,16426332819,everyone,1,1:53,2:56,1:53,strava://segments/34463750/leaderboard,overall,All-Time,64841306.0,Bartek Ryczek,https://dgalywyr863hv.cloudfront.net/pictures/...,6 efforts in the last 90 days,6,6 efforts,4 efforts,strava://segments/34463750/local_legend?catego...,NaN
2,17455116,3,Folwarczna śluza do swojczyckiego,Run,1533.3,0.1,0.8,115.0,112.9,"[51.104281, 17.124089]","[51.114371, 17.109618]",https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,0,Wrocław,Województwo dolnośląskie,Poland,False,False,False,2018-04-22T13:45:42Z,2021-05-15T02:20:56Z,2.1,26055,3021,3,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,s17455116,wh|vHopogBGX[b@]v@[\IVuAfBeCjD}ApCs@z@oBbDi@n@...,3,498,2021-10-25,everyone,6165241355,everyone,38,5:03,6:10,5:03,strava://segments/17455116/leaderboard,overall,All-Time,105315738.0,Marcin Przepiórkowski,https://lh3.googleusercontent.com/a/ACg8ocJUZd...,26 efforts in the last 90 days,26,26 efforts,14 efforts,strava://segments/17455116/local_legend?catego...,NaN
3,8544270,3,Wschodni ~2.8k Lap,Run,2784.3,0.0,11.0,101.2,99.0,"[51.085002, 17.075916]","[51.084648, 17.075653]",https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,0,Wrocław,Województwo dolnośląskie,Poland,False,False,False,2014-11-22T16:03:40Z,2021-05-19T08:03:05Z,2.0,15012,1442,30,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,https://d3o5xota0a1fcr.cloudfront.net/v6/chart...,s8544270,gpxvHmcfgBcA_ES{A]qAS_AMw@Gu@a@wBUgBEo@Y}AKyBI...,3,979,2023-01-11,everyone,8373136865,everyone,4,9:24,13:04,9:24,strava://segments/8544270/leaderboard,overall,All-Time,85087334.0,Paweł Rakowiecki,https://graph.facebook.com/10225563450261374/p...,42 efforts in the last 90 days,42,42 efforts,20 efforts,strava://segments/8544270/local_legend?categor...,NaN
4,6625302,3,Eisenbahnbrücke - Garstnerbad,Run,939.4,-0.7,3.0,310

### Set `bronze.segment_details` types map

In [40]:
segment_details_dtype_map = {
    "id": BigInteger,
    "resource_state": Integer,
    "name": Text,
    "activity_type": Text,
    "distance": Float,
    "average_grade": Float,
    "maximum_grade": Float,
    "elevation_high": Float,
    "elevation_low": Float,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "elevation_profile": Text,
    "climb_category": Integer,
    "city": Text,
    "state": Text,
    "country": Text,
    "private": Boolean,
    "hazardous": Boolean,
    "starred": Boolean,
    "created_at": Text,
    "updated_at": Text,
    "total_elevation_gain": Float,
    "effort_count": Integer,
    "athlete_count": Integer,
    "star_count": Integer,
    "elevation_profiles_light_url": Text,
    "elevation_profiles_dark_url": Text,
    "map_id": Text,
    "map_polyline": Text,
    "map_resource_state": Integer,
    "athlete_segment_stats_pr_elapsed_time": Text,
    "athlete_segment_stats_pr_date": Text,
    "athlete_segment_stats_pr_visibility": Text,
    "athlete_segment_stats_pr_activity_id": BigInteger,
    "athlete_segment_stats_pr_activity_visibility": Text,
    "athlete_segment_stats_effort_count": Integer,
    "xoms_kom": Text,
    "xoms_qom": Text,
    "xoms_overall": Text,
    "xoms_destination_href": Text,
    "xoms_destination_type": Text,
    "xoms_destination_name": Text,
    "local_legend_athlete_id": BigInteger,
    "local_legend_title": Text,
    "local_legend_profile": Text,
    "local_legend_effort_description": Text,
    "local_legend_effort_count": Integer,
    "local_legend_effort_counts_overall": Text,
    "local_legend_effort_counts_female": Text,
    "local_legend_destination": Text
}

### Save `bronze.segment_details` to database

In [41]:
if UPDATE_SEGMENT:
  table_full_name = f"{TARGET_B_SCHEMA}.{SEGMENTS_DETAILS_B_TABLE}"
  logging.warning(f"Whole table {table_full_name} will be overwritten.")
  segments_df_new.to_sql(
    SEGMENTS_DETAILS_B_TABLE, 
    engine, 
    schema=TARGET_B_SCHEMA, 
    if_exists="replace", 
    index=False, 
    method="multi", 
    chunksize=1000, 
    dtype=segment_details_dtype_map
  )

2025-12-09 14:40:06,184 | WARNING | Whole table bronze.segments_details will be overwritten.


### Set `bronze.activities_zones` types map

In [42]:
activities_zones_dtype_map = {
    "score": Float,
    "distribution_buckets": JSONB,
    "type": Text,
    "resource_state": Text,
    "sensor_based": Boolean,
    "points": Float,
    "custom_zones": Boolean,
    "activity_id": BigInteger,
    "id": Text
}

### Save `bronze.activities_zones` to database

In [43]:
if zones_df_new.empty:
  logging.info('No new zones to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=ZONES_B_TABLE, schema=TARGET_B_SCHEMA):
    zones_df_new.head(0).to_sql(
      ZONES_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_zones_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=ZONES_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE zones_stg
        AS TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    zones_df_new.to_sql(
      "zones_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_zones_dtype_map
      )
    
    # Prepare upsert
    stg_cols = zones_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in zones_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{ZONES_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM zones_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS zones_stg;'))
  
    logging.info('Activities zones saved to PostgreSQL.')

2025-12-09 14:40:07,074 | INFO | Activities zones saved to PostgreSQL.
